In [2]:
import os
import glob

# 1. SETUP PATHS
INPUT_DIR = '/home/fetalusr1/Fetal-Head-Segmentation-master/Param/'
RESULTS_ROOT = './Results/'

# 2. FIND ALL FILES
# This finds every .nii file in your Param folder
all_nii_files = sorted(glob.glob(os.path.join(INPUT_DIR, '*.nii')))

print(f"📂 Found {len(all_nii_files)} volumes in {INPUT_DIR}")
print(f"📁 Subfolders will be created in: {os.path.abspath(RESULTS_ROOT)}")

# Display the first few to verify
for f in all_nii_files[:5]:
    print(f"  -> Ready: {os.path.basename(f)}")

📂 Found 36 volumes in /home/fetalusr1/Fetal-Head-Segmentation-master/Param/
📁 Subfolders will be created in: /home/fetalusr1/Fetal-Head-Segmentation-master/Results
  -> Ready: IMG_20250329_11.nii
  -> Ready: IMG_20250329_12_1.nii
  -> Ready: IMG_20250329_12_2.nii
  -> Ready: IMG_20250329_13_1.nii
  -> Ready: IMG_20250329_13_2.nii


In [21]:
import os
# optimizing memory allocation to reduce fragmentation
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"
print("✅ Memory fragmentation rules applied.")

✅ Memory fragmentation rules applied.


In [22]:
import os
import torch

# 1. Help PyTorch manage fragmented memory
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"

# 2. Clear any lingering cache
torch.cuda.empty_cache()

print(f"✅ Memory settings applied. Free memory: {torch.cuda.mem_get_info()[0] / 1024**3:.2f} GB")

✅ Memory settings applied. Free memory: 43.60 GB


In [23]:
import os
import sys

# Get the path to your current environment
conda_prefix = sys.prefix
lib_path = os.path.join(conda_prefix, 'lib')

# Force this path to the front of the line
os.environ['LD_LIBRARY_PATH'] = f"{lib_path}:{os.environ.get('LD_LIBRARY_PATH', '')}"

print(f"✅ Forced Library Path: {lib_path}")

✅ Forced Library Path: /home/fetalusr1/miniconda3/envs/fetal_project/lib


In [3]:
from PIL import Image
import torch
import numpy as np
from modeling.BaseModel import BaseModel
from modeling import build_model
from utilities.distributed import init_distributed
from utilities.arguments import load_opt_from_config_files
from utilities.constants import BIOMED_CLASSES
import matplotlib.pyplot as plt
from inference_utils.inference import interactive_infer_image
from inference_utils.output_processing import check_mask_stats
from inference_utils.processing_utils import process_intensity_image
from inference_utils.processing_utils import read_nifti
import nibabel as nib
import pandas as pd
import SimpleITK as sitk
from skimage.measure import regionprops, label
from skimage.transform import resize
import glob

# ==========================================
# BATCH CONFIGURATION
# ==========================================
INPUT_DIR = '/home/fetalusr1/Fetal-Head-Segmentation-master/Param/'
RESULTS_ROOT = './Results/'
os.makedirs(RESULTS_ROOT, exist_ok=True)

# Find all files in the Param folder
all_nii_files = sorted(glob.glob(os.path.join(INPUT_DIR, '*.nii')))
print(f"📂 Found {len(all_nii_files)} files. Starting Batch Process...")

# ==========================================
# START THE BIG LOOP
# ==========================================
for image_path in all_nii_files:
    # Extract the ID (e.g., 13_1) for folder naming
    filename = os.path.basename(image_path)
    parts = filename.split('_')
    # Extracts the "13_1" part from "IMG_20250329_13_1.nii"
    file_id = "_".join(parts[2:]).replace('.nii', '')
    
    # Create the Subfolder for this specific patient
    patient_dir = os.path.join(RESULTS_ROOT, file_id)
    os.makedirs(patient_dir, exist_ok=True)
    
    print(f"\n▶️ Processing Patient: {file_id}")
    
    # RESET LISTS FOR EVERY PATIENT 
    # (This ensures 1_1 data doesn't mix with 15_2)
    out_probs = []
    predicted_masks = []
    
    # Your next block (Model Loading & Inference Loop) will go here...

📂 Found 36 files. Starting Batch Process...

▶️ Processing Patient: 11

▶️ Processing Patient: 12_1

▶️ Processing Patient: 12_2

▶️ Processing Patient: 13_1

▶️ Processing Patient: 13_2

▶️ Processing Patient: 13_3

▶️ Processing Patient: 14_1

▶️ Processing Patient: 14_2

▶️ Processing Patient: 14_3

▶️ Processing Patient: 15_1

▶️ Processing Patient: 15_2

▶️ Processing Patient: 1_1

▶️ Processing Patient: 1_2

▶️ Processing Patient: 1_3

▶️ Processing Patient: 2_1

▶️ Processing Patient: 2_2

▶️ Processing Patient: 2_3

▶️ Processing Patient: 3

▶️ Processing Patient: 3_1

▶️ Processing Patient: 3_2

▶️ Processing Patient: 3_3

▶️ Processing Patient: 4_1

▶️ Processing Patient: 4_2

▶️ Processing Patient: 5_1

▶️ Processing Patient: 5_2

▶️ Processing Patient: 5_3

▶️ Processing Patient: 5_4

▶️ Processing Patient: 6_1

▶️ Processing Patient: 6_2

▶️ Processing Patient: 7_1

▶️ Processing Patient: 7_2

▶️ Processing Patient: 7_3

▶️ Processing Patient: 8

▶️ Processing Patient: 9_1

## Loading the Finetuned BiomedParse model

In [25]:
# ==========================================
# 1. LOAD MODEL (ONCE)
# ==========================================
# Build model config
opt = load_opt_from_config_files(["configs/biomedparse_inference.yaml"])
opt = init_distributed(opt)

# Load model from pretrained weights
finetuned_pth = '/home/fetalusr1/Fetal-Head-Segmentation-master/model_state_dict.pt' 

model = BaseModel(opt, build_model(opt)).from_pretrained(pretrained=finetuned_pth).eval().cuda()

with torch.no_grad():
    model.model.sem_seg_head.predictor.lang_encoder.get_text_embeddings(BIOMED_CLASSES + ["background"], is_eval=True)

print("✅ Model loaded and ready for batch processing.")

# ==========================================
# 2. START THE BATCH LOOP
# ==========================================
for image_path in all_nii_files:
    # Extract ID (e.g., 1_1) and create subfolder
    filename = os.path.basename(image_path)
    parts = filename.split('_')
    file_id = "_".join(parts[2:]).replace('.nii', '')
    
    patient_dir = os.path.join(RESULTS_ROOT, file_id)
    os.makedirs(patient_dir, exist_ok=True)
    
    print(f"\n▶️ Processing: {file_id}")
    
    # RESET LISTS FOR THIS SPECIFIC PATIENT
    out_probs = []
    predicted_masks = []
    
    # LOAD VOLUME DATA
    vol = nib.load(image_path)
    vol_data = vol.get_fdata()
    text_prompt = ['fetal head']
    
    # Initialize volume to store all prediction masks for this patient
    pred_volume = np.zeros((vol_data.shape[0], vol_data.shape[1], vol_data.shape[2]))
    
    # Proceed to your inference loop (next code block)...

$UNUSED$ criterion.empty_weight, Ckpt Shape: torch.Size([17])


✅ Model loaded and ready for batch processing.

▶️ Processing: 11

▶️ Processing: 12_1

▶️ Processing: 12_2

▶️ Processing: 13_1

▶️ Processing: 13_2

▶️ Processing: 13_3

▶️ Processing: 14_1

▶️ Processing: 14_2

▶️ Processing: 14_3

▶️ Processing: 15_1

▶️ Processing: 15_2

▶️ Processing: 1_1

▶️ Processing: 1_2

▶️ Processing: 1_3

▶️ Processing: 2_1

▶️ Processing: 2_2

▶️ Processing: 2_3

▶️ Processing: 3

▶️ Processing: 3_1

▶️ Processing: 3_2

▶️ Processing: 3_3

▶️ Processing: 4_1

▶️ Processing: 4_2

▶️ Processing: 5_1

▶️ Processing: 5_2

▶️ Processing: 5_3

▶️ Processing: 5_4

▶️ Processing: 6_1

▶️ Processing: 6_2

▶️ Processing: 7_1

▶️ Processing: 7_2

▶️ Processing: 7_3

▶️ Processing: 8

▶️ Processing: 9_1

▶️ Processing: 9_2

▶️ Processing: 9_3


## Utilities

### Post-processing Utility

In [29]:
def get_segmentation_masks(original_image, segmentation_masks, texts, rotate=0):
    ''' Plot a list of segmentation mask over an image showing only the segmented region. '''
    original_image = original_image[:, :, :3]
    segmented_images = []
    for i, mask in enumerate(segmentation_masks):
        segmented_image = original_image.copy()
        segmented_image[mask <= 0.5] = [0, 0, 0]
        segmented_images.append(segmented_image)
    return segmented_images

def inference_nifti(file_path, text_prompts, is_CT, slice_idx, site=None, HW_index=(0, 1), channel_idx=None, rotate=0):
    # Load the specific slice
    image = read_nifti(file_path, is_CT, slice_idx, site=site, HW_index=HW_index, channel_idx=channel_idx)
    
    # Run the model inference
    pred_mask, out_prob = interactive_infer_image(model, Image.fromarray(image), text_prompts)
    
    # Update our tracking lists for interpolation later
    predicted_masks.append(pred_mask)
    out_probs.append(out_prob)
    
    # --- THIS WAS THE MISSING LINE ---
    # We must generate the list of images to return it
    segmented_images = get_segmentation_masks(image, pred_mask, text_prompts, rotate=rotate)
    # ---------------------------------
    
    return image, pred_mask, segmented_images

def process_predicted_volume(volume_data, threshold_factor=0.35, output_prefix='processed'):
    """ Process the predicted volume to filter based on ellipse measurements. """
    data = volume_data
    print(f"Processing volume with shape: {data.shape}")
    
    results = []
    z_0 = data.shape[2] // 2 
    
    for i in range(data.shape[2]):
        slice_data = data[:, :, i]
        if np.sum(slice_data) == 0:
            continue
            
        slice_bin = np.where(slice_data > 0, 1, 0).astype(np.uint8)
        slice_bin_filled = sitk.BinaryFillhole(sitk.GetImageFromArray(slice_bin))
        slice_bin_filled = sitk.GetArrayFromImage(slice_bin_filled)
        
        labeled_image = label(slice_bin_filled)
        props = regionprops(labeled_image)
        
        for prop in props:
            results.append({
                'slice_index': i,
                'major_axis_length': prop.major_axis_length,
                'minor_axis_length': prop.minor_axis_length,
                'centroid_x': prop.centroid[1],
                'centroid_y': prop.centroid[0],
                'orientation': prop.orientation,
                'area': prop.area
            })
    
    df_results = pd.DataFrame(results)
    
    standard_slice_data = df_results[df_results['slice_index'] == z_0]
    if standard_slice_data.empty:
        major_axis_length_std = df_results['major_axis_length'].median()
        minor_axis_length_std = df_results['minor_axis_length'].median()
    else:
        major_axis_length_std = standard_slice_data['major_axis_length'].values[0]
        minor_axis_length_std = standard_slice_data['minor_axis_length'].values[0]
    
    major_axis_length_threshold = major_axis_length_std * (1 - threshold_factor)
    minor_axis_length_threshold = minor_axis_length_std * (1 - threshold_factor)
    
    filtered_df = df_results[
        (df_results['major_axis_length'] >= major_axis_length_threshold) &
        (df_results['minor_axis_length'] >= minor_axis_length_threshold)
    ]
    
    filtered_df = filtered_df.loc[filtered_df.groupby('slice_index')['major_axis_length'].idxmax()]
    
    filtered_volume = np.zeros_like(data)
    for slice_idx in range(data.shape[2]):
        if slice_idx in filtered_df['slice_index'].unique():
            filtered_volume[:, :, slice_idx] = data[:, :, slice_idx]
    
    return filtered_volume, filtered_df

### Interpolation Utility

In [30]:
import scipy.ndimage as ndimage

def interpolate_blank_slices(image_path, processed_volume, blank_slices, predicted_masks, delta=1):
    """ Interpolate blank slices in the processed volume using the previous slice. """
    vol_data = nib.load(image_path).get_fdata()
    central_slice = vol_data.shape[2] // 2
    
    # BATCH FIX: Ensure predicted_masks list is long enough for this specific volume
    while len(predicted_masks) < vol_data.shape[2]:
        predicted_masks.append([np.zeros((vol_data.shape[0], vol_data.shape[1]))])

    for slice_idx in blank_slices:
        prev_slice_idx = slice_idx - delta
        if prev_slice_idx < 0 or prev_slice_idx >= len(predicted_masks):
            continue
            
        prev_mask = predicted_masks[prev_slice_idx][0]
        predicted_masks[slice_idx] = [prev_mask.copy()] 
        
        if np.sum(prev_mask) == 0:
            continue
            
        # Scale the mask based on position relative to center
        if slice_idx < central_slice: 
            new_mask = prev_mask * 1.005
        else:
            new_mask = prev_mask * 0.995
        
        image = read_nifti(image_path, is_CT=False, slice_idx=slice_idx, site=None, HW_index=(0, 1), channel_idx=None)
        new_segmented_image = get_segmentation_masks(image, [new_mask], ['fetal head'], rotate=0)[0]
        
        if len(new_segmented_image.shape) == 3:
            gray_mask = np.mean(new_segmented_image, axis=2)
        else:
            gray_mask = new_segmented_image
        
        processed_volume[:, :, slice_idx] = resize(gray_mask, (vol_data.shape[0], vol_data.shape[1]), preserve_range=True)
    
    return processed_volume

def save_patient_results(file_id, patient_dir, original_nii, pred_vol, proc_vol, interp_vol, vol_data):
    """ 
    Exact replication of your saving logic, modified to save into subfolders.
    Includes the 'Conservative Dilation' fix for the skull.
    """
    # 1. Save Raw, Filtered, and Interpolated .nii files
    nib.save(nib.Nifti1Image(pred_vol, original_nii.affine, original_nii.header), 
             os.path.join(patient_dir, f'segmentation_RAW_{file_id}.nii.gz'))
    
    nib.save(nib.Nifti1Image(proc_vol, original_nii.affine, original_nii.header), 
             os.path.join(patient_dir, f'segmentation_fil_{file_id}.nii.gz'))
    
    nib.save(nib.Nifti1Image(interp_vol, original_nii.affine, original_nii.header), 
             os.path.join(patient_dir, f'segmentation_inter_{file_id}.nii.gz'))

    # 2. Final Corrected/Preserved (Dilation Logic)
    current_mask = interp_vol > 0
    structure = ndimage.generate_binary_structure(3, 1)
    dilated_mask = ndimage.binary_dilation(current_mask, structure=structure, iterations=3)
    dilated_mask = ndimage.median_filter(dilated_mask.astype(np.float32), size=3)
    
    final_output = vol_data * dilated_mask
    final_nii = nib.Nifti1Image(final_output.astype(np.float32), original_nii.affine, original_nii.header)
    
    save_path = os.path.join(patient_dir, f'{file_id}_CORRECTED_FINAL.nii.gz')
    nib.save(final_nii, save_path)
    
    return save_path

## Working

In [31]:
# ==========================================
# FINAL ASSEMBLY: BATCH EXECUTION LOOP
# ==========================================

# Dictionary to store the final report for all patients
all_blank_reports = {}

for image_path in all_nii_files:
    # 1. EXTRACT FILE ID AND SETUP FOLDERS
    filename = os.path.basename(image_path)
    parts = filename.split('_')
    # Extracts "1_1" from "IMG_20250329_1_1.nii"
    file_id = "_".join(parts[2:]).replace('.nii', '')
    
    # Create the specific Subfolder: Results/1_1/
    patient_dir = os.path.join(RESULTS_ROOT, file_id)
    os.makedirs(patient_dir, exist_ok=True)
    
    print(f"\n🚀 STARTING BATCH PROCESS FOR: {file_id}")
    
    # RESET LISTS FOR THE NEW PATIENT
    out_probs = []
    predicted_masks = []
    
    # 2. INITIALIZE VOLUME (Your Exact Logic)
    text_prompt = ['fetal head']
    vol = nib.load(image_path)
    vol_data = vol.get_fdata()
    pred_volume = np.zeros((vol_data.shape[0], vol_data.shape[1], vol_data.shape[2]))

    # 3. INFERENCE LOOP
    for slice_idx in range(vol_data.shape[2]):
        image, pred_mask, segmentation_mask = inference_nifti(
            image_path, text_prompt, is_CT=False, slice_idx=slice_idx, site=None, rotate=0
        )
        
        # Grayscale conversion
        if len(segmentation_mask[0].shape) == 3:
            gray_mask = np.mean(segmentation_mask[0], axis=2)
        else:
            gray_mask = segmentation_mask[0]
        
        # Store in volume with resize
        pred_volume[:, :, slice_idx] = resize(gray_mask, (vol_data.shape[0], vol_data.shape[1]), preserve_range=True)

    # 4. POST-PROCESSING (Your Exact Logic)
    processed_volume, filtered_measurements = process_predicted_volume(
        pred_volume, 
        threshold_factor=0.4, 
        output_prefix=file_id
    )

    # 5. BLANK SLICE DETECTION (The Report You Requested)
    if not filtered_measurements.empty:
        first_filtered_slice = min(filtered_measurements['slice_index'].unique())
        last_filtered_slice = max(filtered_measurements['slice_index'].unique())
        
        blank_slices = []
        for slice_idx in range(first_filtered_slice, last_filtered_slice + 1):
            if np.sum(processed_volume[:, :, slice_idx]) == 0:
                blank_slices.append(slice_idx)
        
        # Print the specific line for this patient
        print(f"Blank slices for {file_id} : {blank_slices}")
        all_blank_reports[file_id] = blank_slices
    else:
        print(f"⚠️ No valid regions found for {file_id}. Skipping interpolation.")
        blank_slices = []
        all_blank_reports[file_id] = "No valid regions found"

    # 6. INTERPOLATION
    interpolated_volume = interpolate_blank_slices(
        image_path, processed_volume, blank_slices, predicted_masks, delta=1
    )

    # 7. CONSERVATIVE DILATION (The Skull Fix)
    current_mask = interpolated_volume > 0
    print(f"🛡️ Applying conservative dilation for {file_id}...")
    structure = ndimage.generate_binary_structure(3, 1)
    dilated_mask = ndimage.binary_dilation(current_mask, structure=structure, iterations=3)
    dilated_mask = ndimage.median_filter(dilated_mask.astype(np.float32), size=3)
    
    final_output = vol_data * dilated_mask

    # 8. SAVE RESULTS INTO THE SUBFOLDER
    # Raw Prediction
    nib.save(nib.Nifti1Image(pred_volume, vol.affine, vol.header), 
             os.path.join(patient_dir, f'segmentation_RAW_{file_id}.nii.gz'))
    
    # Processed (Filtered) Prediction
    nib.save(nib.Nifti1Image(processed_volume, vol.affine, vol.header), 
             os.path.join(patient_dir, f'segmentation_fil_{file_id}.nii.gz'))
    
    # Interpolated Prediction
    nib.save(nib.Nifti1Image(interpolated_volume, vol.affine, vol.header), 
             os.path.join(patient_dir, f'segmentation_inter_{file_id}.nii.gz'))
    
    # FINAL CORRECTED RESULT
    final_nii = nib.Nifti1Image(final_output.astype(np.float32), vol.affine, vol.header)
    nib.save(final_nii, os.path.join(patient_dir, f'{file_id}_Corrected.nii.gz'))
    
    print(f"✅ Success. All results for {file_id} saved in {patient_dir}")

# ==========================================
# FINAL SUMMARY PRINT
# ==========================================
print("\n" + "="*40)
print("🏁 BATCH COMPLETED: FINAL BLANK SLICE REPORT")
print("="*40)
for fid, report in all_blank_reports.items():
    print(f"Blank slices for {fid} : {report}")


🚀 STARTING BATCH PROCESS FOR: 11
Processing volume with shape: (205, 157, 247)
Blank slices for 11 : [64, 171, 172, 173, 174, 175, 176, 177, 210, 211, 212, 213, 221, 222, 223, 224, 225, 226]
🛡️ Applying conservative dilation for 11...
✅ Success. All results for 11 saved in ./Results/11

🚀 STARTING BATCH PROCESS FOR: 12_1


/home/fetalusr1/miniconda3/envs/fetal_project/lib/python3.8/site-packages/detectron2/structures/image_list.py:88: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  max_size = (max_size + (stride - 1)) // stride * stride
/home/fetalusr1/Fetal-Head-Segmentation-master/modeling/modules/position_encoding.py:41: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mo

Processing volume with shape: (209, 184, 207)
Blank slices for 12_1 : [145, 151, 153, 174, 175]
🛡️ Applying conservative dilation for 12_1...
✅ Success. All results for 12_1 saved in ./Results/12_1

🚀 STARTING BATCH PROCESS FOR: 12_2


/home/fetalusr1/miniconda3/envs/fetal_project/lib/python3.8/site-packages/detectron2/structures/image_list.py:88: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  max_size = (max_size + (stride - 1)) // stride * stride
/home/fetalusr1/Fetal-Head-Segmentation-master/modeling/modules/position_encoding.py:41: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mo

Processing volume with shape: (215, 169, 218)
Blank slices for 12_2 : [31, 32, 33, 34, 35, 36, 37, 38, 39, 42, 43, 150, 151, 153, 155, 156]
🛡️ Applying conservative dilation for 12_2...
✅ Success. All results for 12_2 saved in ./Results/12_2

🚀 STARTING BATCH PROCESS FOR: 13_1


...

Processing volume with shape: (231, 154, 223)
Blank slices for 9_1 : [18, 19, 20, 21, 22, 23, 24, 42, 43, 50, 51, 52, 53, 54, 55, 136, 137, 154, 155, 156, 157, 158, 159]
🛡️ Applying conservative dilation for 9_1...
✅ Success. All results for 9_1 saved in ./Results/9_1

🚀 STARTING BATCH PROCESS FOR: 9_2


/home/fetalusr1/miniconda3/envs/fetal_project/lib/python3.8/site-packages/detectron2/structures/image_list.py:88: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  max_size = (max_size + (stride - 1)) // stride * stride
/home/fetalusr1/Fetal-Head-Segmentation-master/modeling/modules/position_encoding.py:41: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mo

Processing volume with shape: (208, 137, 278)
Blank slices for 9_2 : [58, 59, 60, 61, 62, 70, 77, 78, 79, 80, 81, 82, 83, 173, 175, 176, 177, 178, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 226, 227]
🛡️ Applying conservative dilation for 9_2...
✅ Success. All results for 9_2 saved in ./Results/9_2

🚀 STARTING BATCH PROCESS FOR: 9_3


/home/fetalusr1/miniconda3/envs/fetal_project/lib/python3.8/site-packages/detectron2/structures/image_list.py:88: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  max_size = (max_size + (stride - 1)) // stride * stride
/home/fetalusr1/Fetal-Head-Segmentation-master/modeling/modules/position_encoding.py:41: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mo

Processing volume with shape: (231, 154, 223)
Blank slices for 9_3 : [18, 19, 20, 21, 22, 23, 24, 42, 43, 50, 51, 52, 53, 54, 55, 136, 137, 154, 155, 156, 157, 158, 159]
🛡️ Applying conservative dilation for 9_3...
✅ Success. All results for 9_3 saved in ./Results/9_3

🏁 BATCH COMPLETED: FINAL BLANK SLICE REPORT
Blank slices for 11 : [64, 171, 172, 173, 174, 175, 176, 177, 210, 211, 212, 213, 221, 222, 223, 224, 225, 226]
Blank slices for 12_1 : [145, 151, 153, 174, 175]
Blank slices for 12_2 : [31, 32, 33, 34, 35, 36, 37, 38, 39, 42, 43, 150, 151, 153, 155, 156]
Blank slices for 13_1 : [144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227]
Blank slices for 13_2 : [59, 60, 145, 146, 147, 153, 154, 156, 157, 176, 178, 179]
Blank slices for 13_3 : [1, 2, 3, 4, 5, 6, 7, 8, 10, 12, 13, 14, 15, 16, 17, 18, 1

In [ ]:
!ls Param/

IMG_20250329_1_1.nii   IMG_20250329_15_1.nii  IMG_20250329_5_2.nii
IMG_20250329_11.nii    IMG_20250329_15_2.nii  IMG_20250329_5_3.nii
IMG_20250329_12_1.nii  IMG_20250329_2_1.nii   IMG_20250329_5_4.nii
IMG_20250329_12_2.nii  IMG_20250329_2_2.nii   IMG_20250329_6_1.nii
IMG_20250329_1_2.nii   IMG_20250329_2_3.nii   IMG_20250329_6_2.nii
IMG_20250329_13_1.nii  IMG_20250329_3_1.nii   IMG_20250329_7_1.nii
IMG_20250329_13_2.nii  IMG_20250329_3_2.nii   IMG_20250329_7_2.nii
IMG_20250329_13_3.nii  IMG_20250329_3_3.nii   IMG_20250329_7_3.nii
IMG_20250329_1_3.nii   IMG_20250329_3.nii     IMG_20250329_8.nii
IMG_20250329_14_1.nii  IMG_20250329_4_1.nii   IMG_20250329_9_1.nii
IMG_20250329_14_2.nii  IMG_20250329_4_2.nii   IMG_20250329_9_2.nii
IMG_20250329_14_3.nii  IMG_20250329_5_1.nii   IMG_20250329_9_3.nii


In [32]:
import os
import shutil
import glob

# 1. SETUP PATHS
INPUT_DIR = '/home/fetalusr1/Fetal-Head-Segmentation-master/Param/'
RESULTS_ROOT = './Results/'

# 2. GET LIST OF COMPLETED SUBFOLDERS
# This looks at every folder inside your Results directory
completed_subfolders = [f for f in os.listdir(RESULTS_ROOT) if os.path.isdir(os.path.join(RESULTS_ROOT, f))]

print(f"📂 Found {len(completed_subfolders)} result folders. Starting copy...")

copy_count = 0
for folder_id in completed_subfolders:
    # Target folder: ./Results/1_1/
    target_dir = os.path.join(RESULTS_ROOT, folder_id)
    
    # Construct the expected original filename 
    # (Matches your logic: IMG_20250329_ + ID + .nii)
    original_filename = f"IMG_20250329_{folder_id}.nii"
    source_path = os.path.join(INPUT_DIR, original_filename)
    
    # Check if the original file exists in Param/
    if os.path.exists(source_path):
        destination_path = os.path.join(target_dir, original_filename)
        
        # copy2 preserves the original file's timestamp
        shutil.copy2(source_path, destination_path)
        copy_count += 1
        print(f"✅ Copied {original_filename} to {target_dir}")
    else:
        print(f"⚠️ Warning: Could not find original file for {folder_id} at {source_path}")

print(f"\n🏁 Finished! Copied {copy_count} original files into their respective subfolders.")

📂 Found 36 result folders. Starting copy...
✅ Copied IMG_20250329_3.nii to ./Results/3
✅ Copied IMG_20250329_14_2.nii to ./Results/14_2
✅ Copied IMG_20250329_4_1.nii to ./Results/4_1
✅ Copied IMG_20250329_6_1.nii to ./Results/6_1
✅ Copied IMG_20250329_6_2.nii to ./Results/6_2
✅ Copied IMG_20250329_3_2.nii to ./Results/3_2
✅ Copied IMG_20250329_5_2.nii to ./Results/5_2
✅ Copied IMG_20250329_2_2.nii to ./Results/2_2
✅ Copied IMG_20250329_9_1.nii to ./Results/9_1
✅ Copied IMG_20250329_8.nii to ./Results/8
✅ Copied IMG_20250329_2_1.nii to ./Results/2_1
✅ Copied IMG_20250329_12_1.nii to ./Results/12_1
✅ Copied IMG_20250329_11.nii to ./Results/11
✅ Copied IMG_20250329_5_3.nii to ./Results/5_3
✅ Copied IMG_20250329_14_3.nii to ./Results/14_3
✅ Copied IMG_20250329_1_3.nii to ./Results/1_3
✅ Copied IMG_20250329_7_2.nii to ./Results/7_2
✅ Copied IMG_20250329_12_2.nii to ./Results/12_2
✅ Copied IMG_20250329_7_3.nii to ./Results/7_3
✅ Copied IMG_20250329_13_2.nii to ./Results/13_2
✅ Copied IMG_202

In [38]:
import os
import numpy as np
import nibabel as nib
import scipy.ndimage as ndimage
from skimage.transform import resize
from skimage.measure import label, regionprops
import math

# ==========================================
# ROBUST "CENTRAL LOCK" STRATEGY FOR 7_2
# ==========================================
FILE_ID = "7_2"
INPUT_PATH = '/home/fetalusr1/Fetal-Head-Segmentation-master/Param/IMG_20250329_7_2.nii'
OUTPUT_DIR = f'./Results/{FILE_ID}/'
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"🚀 Processing {FILE_ID} with Central Lock Strategy...")

# 1. LOAD & INFERENCE (Standard)
vol = nib.load(INPUT_PATH)
vol_data = vol.get_fdata()
pred_volume = np.zeros(vol_data.shape)

out_probs = []
predicted_masks = []

# Standard Inference Loop
for slice_idx in range(vol_data.shape[2]):
    image, pred_mask, segmentation_mask = inference_nifti(
        INPUT_PATH, ['fetal head'], is_CT=False, slice_idx=slice_idx, site=None, rotate=0
    )
    if len(segmentation_mask[0].shape) == 3:
        gray_mask = np.mean(segmentation_mask[0], axis=2)
    else:
        gray_mask = segmentation_mask[0]
    pred_volume[:, :, slice_idx] = resize(gray_mask, (vol_data.shape[0], vol_data.shape[1]), preserve_range=True)

# 2. FILTERING
processed_volume, filtered_measurements = process_predicted_volume(
    pred_volume, threshold_factor=0.4, output_prefix=FILE_ID
)
if not filtered_measurements.empty:
    first = min(filtered_measurements['slice_index'].unique())
    last = max(filtered_measurements['slice_index'].unique())
    blank_slices = [z for z in range(first, last + 1) if np.sum(processed_volume[:, :, z]) == 0]
else:
    blank_slices = []

interpolated_volume = interpolate_blank_slices(
    INPUT_PATH, processed_volume, blank_slices, predicted_masks, delta=1
)

# ==========================================
# 3. THE FIX: 2D "NECK SNAP" + "CENTRAL LOCK"
# ==========================================
print("🛡️ Applying Central Lock Filtering...")

cleaned_volume = np.zeros_like(interpolated_volume)
current_mask = interpolated_volume > 0

# Calculate Image Center (e.g., 128, 128)
center_y, center_x = current_mask.shape[0] // 2, current_mask.shape[1] // 2

# Process Slice-by-Slice
for z in range(current_mask.shape[2]):
    slice_mask = current_mask[:, :, z]
    if np.sum(slice_mask) == 0: continue

    # A. SNAP THE NECK (Strong 2D Opening)
    # Radius 4 (approx 9x9 pixels) is enough to break the "neck" but keep the brain.
    struct2d = ndimage.generate_binary_structure(2, 1)
    slice_opened = ndimage.binary_opening(slice_mask, structure=struct2d, iterations=4)
    
    # If opening killed everything (rare), revert to original for this slice
    if np.sum(slice_opened) == 0:
        slice_opened = slice_mask

    # B. LABEL COMPONENTS
    labeled_slice, num_features = label(slice_opened, return_num=True)
    
    if num_features == 0:
        continue
        
    # C. FIND THE "HEAD" (Closest to Center + Large Area)
    regions = regionprops(labeled_slice)
    
    best_region = None
    min_dist = float('inf')
    
    for region in regions:
        # Filter 1: Ignore tiny noise specs (Area < 100 pixels)
        if region.area < 100:
            continue
            
        # Filter 2: Calculate Distance to Center
        # The Fetal Head is usually centered. The Shelf is at the bottom.
        centroid_y, centroid_x = region.centroid
        dist = math.sqrt((centroid_y - center_y)**2 + (centroid_x - center_x)**2)
        
        # We prefer the object that is closest to the center
        if dist < min_dist:
            min_dist = dist
            best_region = region
    
    # D. KEEP ONLY THE BEST REGION
    if best_region is not None:
        # Create a blank mask and paint ONLY the best region
        final_slice_mask = (labeled_slice == best_region.label)
        
        # E. RESTORE SIZE (Dilate back what we eroded in step A)
        # We eroded by 4 iterations, so we dilate by 4 to get the original size back
        final_slice_mask = ndimage.binary_dilation(final_slice_mask, structure=struct2d, iterations=4)
        
        cleaned_volume[:, :, z] = final_slice_mask

# ==========================================
# 4. SAVE
# ==========================================
# Smooth the 3D volume slightly to fix any jaggedness between slices
cleaned_volume = ndimage.median_filter(cleaned_volume.astype(np.float32), size=3)

final_output = vol_data * cleaned_volume

final_nii = nib.Nifti1Image(final_output.astype(np.float32), vol.affine, vol.header)
final_nii.header.set_zooms(vol.header.get_zooms())

save_path = os.path.join(OUTPUT_DIR, f'{FILE_ID}_CENTRAL_LOCK.nii.gz')
nib.save(final_nii, save_path)

print(f"✅ CENTRAL LOCK result saved: {save_path}")
print("   (Kept only the central brain object, ignored the bottom shelf)")

🚀 Processing 7_2 with Central Lock Strategy...


/home/fetalusr1/miniconda3/envs/fetal_project/lib/python3.8/site-packages/detectron2/structures/image_list.py:88: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  max_size = (max_size + (stride - 1)) // stride * stride
/home/fetalusr1/Fetal-Head-Segmentation-master/modeling/modules/position_encoding.py:41: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mo

Processing volume with shape: (225, 161, 219)
🛡️ Applying Central Lock Filtering...
✅ CENTRAL LOCK result saved: ./Results/7_2/7_2_CENTRAL_LOCK.nii.gz
   (Kept only the central brain object, ignored the bottom shelf)


In [4]:
import os
import numpy as np
import nibabel as nib

# 1. SETUP
PARAM_DIR = '/home/fetalusr1/Fetal-Head-Segmentation-master/Param/'

# The list of suspect pairs based on your Blank Slice Report
suspect_pairs = [
    ('IMG_20250329_14_1.nii', 'IMG_20250329_14_2.nii'),
    ('IMG_20250329_15_1.nii', 'IMG_20250329_15_2.nii'),
    ('IMG_20250329_2_1.nii',  'IMG_20250329_2_2.nii'),
    ('IMG_20250329_3_1.nii',  'IMG_20250329_3_2.nii'),
    ('IMG_20250329_9_1.nii',  'IMG_20250329_9_3.nii')
]

print(f"{'FILE A':<25} | {'FILE B':<25} | {'STATUS':<15} | {'DIFFERENCE'}")
print("-" * 85)

for filename_a, filename_b in suspect_pairs:
    path_a = os.path.join(PARAM_DIR, filename_a)
    path_b = os.path.join(PARAM_DIR, filename_b)
    
    # Check if files exist
    if not os.path.exists(path_a) or not os.path.exists(path_b):
        print(f"{filename_a:<25} | {filename_b:<25} | ❌ MISSING FILE")
        continue

    # 2. LOAD DATA
    vol_a = nib.load(path_a).get_fdata()
    vol_b = nib.load(path_b).get_fdata()

    # 3. THE MATHEMATICAL CHECK
    # Check 1: Are shapes the same?
    if vol_a.shape != vol_b.shape:
        print(f"{filename_a:<25} | {filename_b:<25} | ⚠️ DIFFERENT SHAPE")
        continue

    # Check 2: Are the values identical?
    # array_equal returns True only if every single pixel matches
    is_identical = np.array_equal(vol_a, vol_b)
    
    # Check 3: Calculate total difference (Sum of all pixel differences)
    diff_sum = np.sum(np.abs(vol_a - vol_b))

    status = "✅ IDENTICAL" if is_identical else "❌ DIFFERENT"
    
    print(f"{filename_a:<25} | {filename_b:<25} | {status:<15} | Diff Sum: {diff_sum:.2f}")

print("-" * 85)

FILE A                    | FILE B                    | STATUS          | DIFFERENCE
-------------------------------------------------------------------------------------
IMG_20250329_14_1.nii     | IMG_20250329_14_2.nii     | ✅ IDENTICAL     | Diff Sum: 0.00
IMG_20250329_15_1.nii     | IMG_20250329_15_2.nii     | ✅ IDENTICAL     | Diff Sum: 0.00
IMG_20250329_2_1.nii      | IMG_20250329_2_2.nii      | ✅ IDENTICAL     | Diff Sum: 0.00
IMG_20250329_3_1.nii      | IMG_20250329_3_2.nii      | ✅ IDENTICAL     | Diff Sum: 0.00
IMG_20250329_9_1.nii      | IMG_20250329_9_3.nii      | ✅ IDENTICAL     | Diff Sum: 0.00
-------------------------------------------------------------------------------------
